In [ ]:
import matplotlib.pyplot as plt
import cyipopt as ipopt
import numpy as np

from vessel_models import VesselModel
from controller import TrajOpt

In [ ]:
# Validating IPOPT
from scipy.optimize import rosen, rosen_der
x0 = [1.3, 0.7, 0.8, 1.9, 1.2]
res = ipopt.minimize_ipopt(rosen, x0, jac=rosen_der)
print(res)

In [ ]:
# Simulation variables
N = 50
dt = 0.1
ns = 6
nu = 3

# Cost matrices
Qvec = 3*np.ones(ns)
Rvec = 1*np.ones(nu)
Qfvec = 10*np.ones(ns)

# Problem Definition
s0 = np.zeros(6)
sf = np.zeros(6)
sf[0] = 5
sf[1] = 5
sf[2] = np.pi/2

# obstacles
num_obstacles = 2
obstacles = np.zeros((num_obstacles, 3))
obstacles[0] = [3, 3, 1]
obstacles[1] = [8, 8, 1]

In [ ]:
model = VesselModel(mismatch=False)
trajoptimizer = TrajOpt(model, N, ns, nu, dt, Qvec, Rvec, Qfvec, s0, sf, obstacles)
trajoptimizer.exec_trajopt()

In [ ]:
print(trajoptimizer.sol)
S, U = trajoptimizer.flat2vec(trajoptimizer.sol.x)
print(np.max(U))
ps = S.T

np.savetxt("dircol_soln.csv", S, delimiter=",")
np.savetxt("dircol_inpu.csv", U, delimiter=",")

fig, axes = plt.subplots(1, 1, figsize=(20, 20))
for obstacle in obstacles:
    obs_x, obs_y, obs_r = obstacle
    circle = plt.Circle((obs_y, obs_x), obs_r, color='r')
    axes.add_patch(circle)
axes.scatter(ps[1], ps[0])
for i in range(len(ps[1])):
    dx = 0.5*np.cos(ps[2,i])
    dy = 0.5*np.sin(ps[2,i])
    axes.arrow(ps[1,i], ps[0, i], dy, dx, head_width=0.05, head_length=0.1, fc='green', ec='green')
axes.set_aspect('equal', adjustable='box')
plt.show()

Sfwd = np.zeros((N,ns))
Sfwd[0] = S[0]
for i in range(N-1):
    Sn = Sfwd[i]
    Un = U[i]
    Sfwd[i+1] = model.rk4(Sn, Un, dt)

ps = Sfwd.T
fig, axes = plt.subplots(1, 1, figsize=(20, 20))
for obstacle in obstacles:
    obs_x, obs_y, obs_r = obstacle
    circle = plt.Circle((obs_y, obs_x), obs_r, color='r')
    axes.add_patch(circle)
axes.scatter(ps[1], ps[0])
axes.set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
Sn = np.zeros(6)
Un = np.zeros(3)
Un[0] = 5
Un[1] = 5

S = np.zeros((N, ns))
S[0] = Sn

U = np.zeros((N-1, nu))
for i in range(N-1):
    U[i] = [40*np.sin(dt*i),np.cos(dt*i),np.cos(dt*i)]

for i in range(N-1):
    S[i+1] = model.rk4(S[i], U[i], dt)

ps = list(zip(*S))
fig, axes = plt.subplots(1, 1, figsize=(20, 5))
axes.scatter(ps[0], ps[1])
plt.show()